In [1]:
#importing all relevant libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import folium
import sklearn 
from sklearn.cluster import KMeans
import requests
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

## Data scrapping and wrangling

In [2]:
# using url and pandas for getting the dataframe
# using pandas to scrap the data from the wikipedia link and store it in data frame. Since the page has '3' tables/dataframe
# accessing the same using indexing
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df= pd.read_html(url, header = 0)
df = df[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [3]:
# Removing the rows having Borough values as Not assigned
# taking a look at the data also reveals that there are no neighbourhoods having 'not Assigned' after having cleaned the 'not assigned' borough 
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True, inplace = True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Using csv file to create data frame

Since getting error while getting data using geopy, using the provided csv file for getting the coordinates

In [4]:
#creating a dataframe from the given location data csv file. 
geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
#using pandas merge feature to match the column values for postal code and create the final required dataframe
finaldf = pd.merge(df, geodata, on = 'Postal Code',how = 'inner')

In [6]:
finaldf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Following an approach similar to the New York Analysis

Visualising the neighborhoods on a map to get a better view of the data

In [7]:
torontomap = folium.Map(location = [43.6532,-79.3832], zoom_start = 10)
#superimposing neighnorhood data from our final dataset
for lat,lng,borough,neighborhood in zip(finaldf['Latitude'],finaldf['Longitude'],finaldf['Borough'],finaldf['Neighborhood']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap)
torontomap

In [8]:
# selecting boroughs that contain the term Toronto
toronto = finaldf[finaldf['Borough'].str.contains('Toronto', na = False)]
toronto.reset_index(drop = True, inplace = True)

In [9]:
toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [10]:
torontomap1 = folium.Map(location = [43.6532,-79.3832], zoom_start = 12)
#superimposing neighnorhood data from our final dataset
for lat,lng, label in zip(toronto['Latitude'],toronto['Longitude'],toronto['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap1)
torontomap1

#### Using Foursquare API

In [11]:
# Defining Foursquare credentials
CLIENT_ID = '1HPEBYY2JHKJRWELE0EKUCI3BHITJT3OASZYVWSQH4LRBKHT'
CLIENT_SECRET = 'Y5DQVNKYZ3NYAM4YEOQUQOS2LYL4THYOFSJ5TH5RUGYAEORA'
VERSION = '20180605'

In [12]:
toronto.loc[4,'Neighborhood']

'The Beaches'

In [13]:
# finding data in the dataframe to test the relevant result
beach_lat = toronto.loc[4,'Latitude']
beach_lon = toronto.loc[4,'Longitude']
name = toronto.loc[4,'Neighborhood']
print (beach_lat)
print (beach_lon)
print (name)

43.67635739999999
-79.2930312
The Beaches


In [14]:
# getting places close to above randomly selected place using Foursquare API request
limit = 100
radius = 300
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET,VERSION,beach_lat,beach_lon,radius,limit)
url1

'https://api.foursquare.com/v2/venues/explore?client_id=1HPEBYY2JHKJRWELE0EKUCI3BHITJT3OASZYVWSQH4LRBKHT&client_secret=Y5DQVNKYZ3NYAM4YEOQUQOS2LYL4THYOFSJ5TH5RUGYAEORA&v=20180605&ll=43.67635739999999,-79.2930312&radius=300&limit=100'

In [15]:
results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5ee70030cf9b553d6820dc6d'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.679057402699996,
    'lng': -79.28930502869432},
   'sw': {'lat': 43.67365739729999, 'lng': -79.29675737130569}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
# getting nearby venues in a dataframe from the API Results
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Glen Stewart Park,Park,43.675278,-79.294647
2,Balmy Beach Playground,Playground,43.676078,-79.290805
3,Best Bathtub in the Beaches,Spa,43.674591,-79.293602


In [18]:
#defining a function to extract nearby places for the entire dataframe
def getVenues(names, latitude, longitude, radius = 300):
    venuelist =[]
    for name, lat,lng in zip(names, latitude, longitude):
        print (name)
        
        url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat, lng, radius,limit)
        
        results = requests.get(url2).json()["response"]['groups'][0]['items']
        
        venuelist.append([(name, lat,lng, v['venue']['name'],
                            v['venue']['location']['lat'], v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venuelist for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return nearby_venues
        

In [19]:
toronto_venues = getVenues(names=toronto['Neighborhood'],
                                   latitude=toronto['Latitude'],
                                   longitude=toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [20]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [21]:
toronto_venues.shape

(900, 7)

In [22]:
#replacing dummies in the data for using ML algorithms
toronto_dummy = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_dummy['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_dummy.head()

,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#grouping the categories according to neighborhood to appply K Means clustering
toronto_grouped = toronto_dummy.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.2,0.2,0.2,0.2,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Christie,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0,0.019231,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.019231,0.000000
7,"Commerce Court, Victoria Hotel",0.0,0.000000,0.0,0.0,0.0,0.0,0.027397,0.000000,0.013699,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000
8,Davisville,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.04,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
#using k means clustering
kclusters = 5
torontoclustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoclustering)
# Manually ran the cluster over multiple values of number of clusters, 5 seems to give a reasonably decent distribution
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 3, 3,
       0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0])

In [25]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped

toronto_merged

,Cluster Labels,Neighborhood,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.2,0.2,0.2,0.2,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,Central Bay Street,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0,Christie,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0,Church and Wellesley,0.0,0.019231,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.019231,0.000000
7,0,"Commerce Court, Victoria Hotel",0.0,0.000000,0.0,0.0,0.0,0.0,0.027397,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000
8,0,Davisville,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.04,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0,Davisville North,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
